# 02 - Preprocesamiento de Datos

## 1. Importaciones y Configuración

In [47]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Preprocesamiento
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Semilla para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Librerías cargadas correctamente")

✓ Librerías cargadas correctamente


## 2. Carga de Datos

In [48]:
# Cargar datos
df = pd.read_csv('train.csv')

print(f"Dimensiones: {df.shape}")
print(f"Registros: {df.shape[0]:,}")
print(f"Columnas: {df.shape[1]}")
print(f"\nColumnas:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

Dimensiones: (692500, 21)
Registros: 692,500
Columnas: 21

Columnas:
  1. ID
  2. PERIODO_ACADEMICO
  3. E_PRGM_ACADEMICO
  4. E_PRGM_DEPARTAMENTO
  5. E_VALORMATRICULAUNIVERSIDAD
  6. E_HORASSEMANATRABAJA
  7. F_ESTRATOVIVIENDA
  8. F_TIENEINTERNET
  9. F_EDUCACIONPADRE
  10. F_TIENELAVADORA
  11. F_TIENEAUTOMOVIL
  12. E_PRIVADO_LIBERTAD
  13. E_PAGOMATRICULAPROPIO
  14. F_TIENECOMPUTADOR
  15. F_TIENEINTERNET.1
  16. F_EDUCACIONMADRE
  17. RENDIMIENTO_GLOBAL
  18. INDICADOR_1
  19. INDICADOR_2
  20. INDICADOR_3
  21. INDICADOR_4


In [51]:
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [52]:
# Guardar copia
ids = df['ID'].copy()
df_original = df.copy()
df_processed = df.copy()
print("✓ Copias creadas")

✓ Copias creadas


## 3. Limpieza: Columnas Duplicadas

In [53]:
# Eliminar F_TIENEINTERNET.1 (duplicada)
if 'F_TIENEINTERNET.1' in df_processed.columns:
    print("Columna duplicada F_TIENEINTERNET.1 encontrada")
    df_processed = df_processed.drop('F_TIENEINTERNET.1', axis=1)
    print("✓ Eliminada")

Columna duplicada F_TIENEINTERNET.1 encontrada
✓ Eliminada


## 4. Análisis de Valores Faltantes

In [54]:
missing = pd.DataFrame({
    'Columna': df_processed.columns,
    'Faltantes': df_processed.isnull().sum(),
    'Porcentaje': (df_processed.isnull().sum() / len(df_processed) * 100).round(2)
}).sort_values('Porcentaje', ascending=False)

missing = missing[missing['Faltantes'] > 0]
if len(missing) > 0:
    print("Valores faltantes:")
    print(missing.to_string(index=False))
else:
    print("✓ Sin valores faltantes")

Valores faltantes:
                    Columna  Faltantes  Porcentaje
           F_TIENEAUTOMOVIL      43623        6.30
            F_TIENELAVADORA      39773        5.74
          F_TIENECOMPUTADOR      38103        5.50
          F_ESTRATOVIVIENDA      32137        4.64
       E_HORASSEMANATRABAJA      30857        4.46
            F_TIENEINTERNET      26629        3.85
           F_EDUCACIONMADRE      23664        3.42
           F_EDUCACIONPADRE      23178        3.35
      E_PAGOMATRICULAPROPIO       6498        0.94
E_VALORMATRICULAUNIVERSIDAD       6287        0.91


## 5. E_VALORMATRICULAUNIVERSIDAD

In [55]:
if 'E_VALORMATRICULAUNIVERSIDAD' in df_processed.columns:
    print("Valores únicos:")
    print(df_processed['E_VALORMATRICULAUNIVERSIDAD'].value_counts())
    print(f"\nFaltantes: {df_processed['E_VALORMATRICULAUNIVERSIDAD'].isnull().sum()}")

Valores únicos:
E_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64

Faltantes: 6287


In [56]:
def convertir_valor_matricula(valor):
    if pd.isna(valor): return np.nan
    elif 'Menos de 500 mil' in valor: return 250000
    elif 'Entre 500 mil y menos de 1 millón' in valor: return 750000
    elif 'Entre 1 millón y menos de 2.5 millones' in valor: return 1750000
    elif 'Entre 2.5 millones y menos de 4 millones' in valor: return 3250000
    elif 'Entre 4 millones y menos de 5.5 millones' in valor: return 4750000
    elif 'Entre 5.5 millones y menos de 7 millones' in valor: return 6250000
    elif 'Más de 7 millones' in valor: return 7500000
    elif 'No pagó matrícula' in valor: return 0
    else: return np.nan

if 'E_VALORMATRICULAUNIVERSIDAD' in df_processed.columns:
    df_processed['E_VALORMATRICULAUNIVERSIDAD'] = df_processed['E_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)
    media = df_processed['E_VALORMATRICULAUNIVERSIDAD'].mean()
    df_processed['E_VALORMATRICULAUNIVERSIDAD'].fillna(media, inplace=True)
    print(f"✓ Convertida. Media: ${media:,.0f}")

✓ Convertida. Media: $2,815,800


## 6. E_HORASSEMANATRABAJA

In [57]:
if 'E_HORASSEMANATRABAJA' in df_processed.columns:
    print("Valores únicos:")
    print(df_processed['E_HORASSEMANATRABAJA'].value_counts())

Valores únicos:
E_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64


In [58]:
def convertir_horas_trabajadas(valor):
    if pd.isna(valor): return np.nan
    valor = str(valor)
    if valor == '0': return 0
    elif 'Entre' in valor:
        try:
            partes = valor.split('y')
            min_val = float(partes[0].split(' ')[-2])
            max_val = float(partes[1].split(' ')[-2])
            return (min_val + max_val) / 2
        except: return np.nan
    elif 'Más de' in valor:
        try: return float(valor.split(' ')[2])
        except: return np.nan
    elif 'Menos de' in valor:
        try: return float(valor.split(' ')[2])
        except: return np.nan
    return np.nan

if 'E_HORASSEMANATRABAJA' in df_processed.columns:
    df_processed['E_HORASSEMANATRABAJA'] = df_processed['E_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)
    df_processed['E_HORASSEMANATRABAJA'].fillna(0, inplace=True)
    print("✓ Convertida")

✓ Convertida


## 7. F_ESTRATOVIVIENDA

In [59]:
if 'F_ESTRATOVIVIENDA' in df_processed.columns:
    estrato_map = {'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}
    df_processed['F_ESTRATOVIVIENDA'] = df_processed['F_ESTRATOVIVIENDA'].map(estrato_map)
    df_processed['F_ESTRATOVIVIENDA'].fillna(0, inplace=True)
    print("✓ Estrato codificado (0-6)")

✓ Estrato codificado (0-6)


## 8. Variables Binarias

In [60]:
binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO']

for col in binary_cols:
    if col in df_processed.columns:
        df_processed[col] = df_processed[col].map({'Si': 1, 'No': 0})
        df_processed[col].fillna(0, inplace=True)
        print(f"✓ {col} codificada (0/1)")

✓ F_TIENEINTERNET codificada (0/1)
✓ F_TIENELAVADORA codificada (0/1)
✓ F_TIENEAUTOMOVIL codificada (0/1)
✓ F_TIENECOMPUTADOR codificada (0/1)
✓ E_PRIVADO_LIBERTAD codificada (0/1)
✓ E_PAGOMATRICULAPROPIO codificada (0/1)


## 9. RENDIMIENTO_GLOBAL (Target)

In [61]:
if 'RENDIMIENTO_GLOBAL' in df_processed.columns:
    target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
    df_processed['RENDIMIENTO_GLOBAL'] = df_processed['RENDIMIENTO_GLOBAL'].map(target_map)
    print("✓ Target codificado (0=bajo, 1=medio-bajo, 2=medio-alto, 3=alto)")
    print(df_processed['RENDIMIENTO_GLOBAL'].value_counts().sort_index())

✓ Target codificado (0=bajo, 1=medio-bajo, 2=medio-alto, 3=alto)
RENDIMIENTO_GLOBAL
0    172987
1    172275
2    171619
3    175619
Name: count, dtype: int64


## 10. Educación Padres (One-Hot)

In [64]:
def to_onehot(x):
    values = np.unique(x)
    indices = [np.argwhere(i == values)[0][0] for i in x]
    return np.eye(len(values))[indices].astype(int)

def replace_column_with_onehot(data, col):
    values = np.unique(data[col])
    onehot_matrix = to_onehot(data[col].values)
    return pd.DataFrame(onehot_matrix, columns=[f"{col}_{values[i]}" for i in range(onehot_matrix.shape[1])], index=data.index)

In [63]:
for col in ['F_EDUCACIONMADRE', 'F_EDUCACIONPADRE']:
    if col in df_processed.columns:
        df_processed[col].fillna('No Aplica', inplace=True)
        onehot_df = replace_column_with_onehot(df_processed[[col]], col)
        df_processed = df_processed.join(onehot_df)
        df_processed.drop(col, axis=1, inplace=True)
        print(f"✓ {col} → {onehot_df.shape[1]} columnas one-hot")

✓ F_EDUCACIONMADRE → 12 columnas one-hot
✓ F_EDUCACIONPADRE → 12 columnas one-hot


## 11. E_PRGM_ACADEMICO (Label Encoding)

In [65]:
if 'E_PRGM_ACADEMICO' in df_processed.columns:
    le = LabelEncoder()
    df_processed['E_PRGM_ACADEMICO'] = le.fit_transform(df_processed['E_PRGM_ACADEMICO'])
    print(f"✓ E_PRGM_ACADEMICO codificado (0-{df_processed['E_PRGM_ACADEMICO'].max()})")

✓ E_PRGM_ACADEMICO codificado (0-947)


## 12. E_PRGM_DEPARTAMENTO (One-Hot)

In [66]:
if 'E_PRGM_DEPARTAMENTO' in df_processed.columns:
    df_processed['E_PRGM_DEPARTAMENTO'].fillna('DESCONOCIDO', inplace=True)
    depto_onehot = replace_column_with_onehot(df_processed[['E_PRGM_DEPARTAMENTO']], 'E_PRGM_DEPARTAMENTO')
    df_processed = df_processed.join(depto_onehot)
    df_processed.drop('E_PRGM_DEPARTAMENTO', axis=1, inplace=True)
    print(f"✓ E_PRGM_DEPARTAMENTO → {depto_onehot.shape[1]} columnas one-hot")

✓ E_PRGM_DEPARTAMENTO → 31 columnas one-hot


## 13. INDICADORES (Variables Numéricas)

In [67]:
indicator_cols = [col for col in df_processed.columns if col.startswith('INDICADOR_')]
if indicator_cols:
    print(f"Indicadores: {indicator_cols}")
    print(df_processed[indicator_cols].describe())
    for col in indicator_cols:
        if df_processed[col].isnull().sum() > 0:
            mediana = df_processed[col].median()
            df_processed[col].fillna(mediana, inplace=True)
            print(f"✓ {col} imputado con mediana")

Indicadores: ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
         INDICADOR_1    INDICADOR_2    INDICADOR_3    INDICADOR_4
count  692500.000000  692500.000000  692500.000000  692500.000000
mean        0.268629       0.259996       0.262087       0.262903
std         0.122130       0.093480       0.058862       0.067944
min         0.000000       0.000000       0.000000       0.000000
25%         0.203000       0.212000       0.254000       0.255000
50%         0.240000       0.271000       0.276000       0.285000
75%         0.314000       0.309000       0.293000       0.303000
max         0.657000       0.487000       0.320000       0.332000


## 14. Outliers

In [68]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return (data[column] < lower) | (data[column] > upper), lower, upper

# Identificar solo columnas REALMENTE numéricas
indicator_cols = [col for col in df_processed.columns if col.startswith('INDICADOR_')]
potential_numeric = ['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA'] + indicator_cols

# Filtrar solo las que sean efectivamente numéricas (int o float)
numeric_features = []
for col in potential_numeric:
    if col in df_processed.columns:
        # Verificar que sea numérica
        if df_processed[col].dtype in ['int64', 'float64']:
            numeric_features.append(col)
        else:
            print(f"⚠️ {col} no es numérica (tipo: {df_processed[col].dtype}), saltando...")

print(f"\nColumnas numéricas para análisis de outliers: {numeric_features}\n")

# Procesar outliers solo en columnas numéricas
for col in numeric_features:
    try:
        mask, lower, upper = detect_outliers_iqr(df_processed, col)
        n_out = mask.sum()
        if n_out > 0:
            df_processed[col] = df_processed[col].clip(lower, upper)
            print(f"✓ {col}: {n_out} outliers capeados")
    except Exception as e:
        print(f"❌ Error procesando {col}: {e}")


Columnas numéricas para análisis de outliers: ['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

✓ INDICADOR_1: 68120 outliers capeados
✓ INDICADOR_2: 34162 outliers capeados
✓ INDICADOR_3: 36488 outliers capeados
✓ INDICADOR_4: 71389 outliers capeados


## 15. Estandarización

In [69]:
# Estandarizar solo las columnas que fueron validadas como numéricas
if numeric_features and len(numeric_features) > 0:
    print(f"Estandarizando {len(numeric_features)} columnas numéricas...")
    print(f"Columnas: {numeric_features}\n")
    
    scaler = StandardScaler()
    df_processed[numeric_features] = scaler.fit_transform(df_processed[numeric_features])
    
    print("✓ Variables numéricas estandarizadas (media=0, std=1)")
    print("\nEstadísticas después de estandarización:")
    print(df_processed[numeric_features].describe().loc[['mean', 'std']].round(6))
else:
    print("⚠️ No hay variables numéricas para estandarizar")

Estandarizando 6 columnas numéricas...
Columnas: ['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

✓ Variables numéricas estandarizadas (media=0, std=1)

Estadísticas después de estandarización:
      E_VALORMATRICULAUNIVERSIDAD  E_HORASSEMANATRABAJA  INDICADOR_1  \
mean                     0.000000              0.000000     0.000000   
std                      1.000001              1.000001     1.000001   

      INDICADOR_2  INDICADOR_3  INDICADOR_4  
mean    -0.000000     0.000000     0.000000  
std      1.000001     1.000001     1.000001  


## 16. Feature Engineering

In [70]:
# Total recursos hogar
recursos = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR']
recursos = [c for c in recursos if c in df_processed.columns]
if recursos:
    df_processed['TOTAL_RECURSOS_HOGAR'] = df_processed[recursos].sum(axis=1)
    print(f"✓ TOTAL_RECURSOS_HOGAR creada")

# Promedio indicadores
if indicator_cols:
    df_processed['PROMEDIO_INDICADORES'] = df_processed[indicator_cols].mean(axis=1)
    print("✓ PROMEDIO_INDICADORES creada")

✓ TOTAL_RECURSOS_HOGAR creada
✓ PROMEDIO_INDICADORES creada


## 17. Preparación Final

In [71]:
# Eliminar ID y PERIODO
for col in ['ID', 'PERIODO_ACADEMICO']:
    if col in df_processed.columns:
        df_processed = df_processed.drop(col, axis=1)

print(f"✓ Dimensiones finales: {df_processed.shape}")
print(f"✓ Valores faltantes: {df_processed.isnull().sum().sum()}")

✓ Dimensiones finales: (692500, 72)
✓ Valores faltantes: 0


## 18. División Train/Validation

In [72]:
if 'RENDIMIENTO_GLOBAL' in df_processed.columns:
    X = df_processed.drop('RENDIMIENTO_GLOBAL', axis=1)
    y = df_processed['RENDIMIENTO_GLOBAL']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
    
    print(f"\n✓ División completada:")
    print(f"  Train: {X_train.shape[0]:,} ({X_train.shape[0]/len(X)*100:.1f}%)")
    print(f"  Val: {X_val.shape[0]:,} ({X_val.shape[0]/len(X)*100:.1f}%)")
else:
    X = df_processed
    y = None


✓ División completada:
  Train: 554,000 (80.0%)
  Val: 138,500 (20.0%)


## 19. Exportación

In [73]:
if y is not None:
    X_train.to_csv('X_train_processed.csv', index=False)
    X_val.to_csv('X_val_processed.csv', index=False)
    y_train.to_csv('y_train.csv', index=False, header=['RENDIMIENTO_GLOBAL'])
    y_val.to_csv('y_val.csv', index=False, header=['RENDIMIENTO_GLOBAL'])
    df_processed.to_csv('data_processed_complete.csv', index=False)
    
    with open('feature_names.txt', 'w') as f:
        f.write(f"# Total: {len(X.columns)} features\n\n")
        for idx, col in enumerate(X.columns, 1):
            f.write(f"{idx}. {col}\n")
    
    print("✓ Archivos exportados:")
    print("  - X_train_processed.csv")
    print("  - X_val_processed.csv")
    print("  - y_train.csv")
    print("  - y_val.csv")
    print("  - data_processed_complete.csv")
    print("  - feature_names.txt")

✓ Archivos exportados:
  - X_train_processed.csv
  - X_val_processed.csv
  - y_train.csv
  - y_val.csv
  - data_processed_complete.csv
  - feature_names.txt
